

# Homework 2 - Recurrent Neural Networks

In this part of the homework we are going to work with Recurrent Neural Networks, in particular GRU. One of the greatest things that Recurrent Neural Networks can do when working with sequences is retaining data from several timesteps in the past. We are going to explore that property by constructing an 'echo' Recurrent Neural Network.

The goal here is to make a model that given a sequence of letters or digits will output that same sequence, but with a certain delay. Let's say the input is a string 'abacaba', we want the model to not output anything for 3 steps (delay length), and then output the original string step by step, except the last 3 characters. So, target output is then 'XXXabac', where 'X' is empty output.

This is similar to [this notebook](https://github.com/Atcold/pytorch-Deep-Learning/blob/master/09-echo_data.ipynb) (which you should refer to when doing this assignment), except we're working not with a binary string, but with a sequence of integers between 0 and some N. In our case N is 26, which is the number of letters in the alphabet.

## Dataset

Let's implement the dataset. In our case, the data is basically infinite, as we can always generate more examples on the fly, so there's no need to load it from disk.

In [2]:
import random
import string

import torch

# Max value of the generated integer. 26 is chosen becuase it's
# the number of letters in English alphabet.
N = 26


def idx_to_onehot(x, k=N+1):
  """ Converts the generated integers to one-hot vectors """
  ones = torch.sparse.torch.eye(k)
  shape = x.shape
  res = ones.index_select(0, x.view(-1).type(torch.int64))
  return res.view(*shape, res.shape[-1])


class EchoDataset(torch.utils.data.IterableDataset):

  def __init__(self, delay=4, seq_length=15, size=1000):
    self.delay = delay
    self.seq_length = seq_length
    self.size = size

  def __len__(self):
    return self.size

  def __iter__(self):
    """ Iterable dataset doesn't have to implement __getitem__.
        Instead, we only need to implement __iter__ to return
        an iterator (or generator).
    """
    for _ in range(self.size):
      seq = torch.tensor([random.choice(range(1, N + 1)) for i in range(self.seq_length)], dtype=torch.int64)
      result = torch.cat((torch.zeros(self.delay), seq[:self.seq_length - self.delay])).type(torch.int64)
      yield seq, result

DELAY = 4
DATASET_SIZE = 200000
ds = EchoDataset(delay=DELAY, size=DATASET_SIZE)

## Model

Now, we want to implement the model. For our purposes, we want to use GRU. The architecture consists of GRU and a decoder. Decoder is responsible for decoding the GRU hidden state to yield a predicting for the next output. The parts you are responsible for filling with your code are marked with `TODO`.

In [3]:
class GRUMemory(torch.nn.Module):

  def __init__(self, hidden_size):
    super().__init__()
    #TODO: initialize your submodules
    self.gru = torch.nn.GRU(N + 1, hidden_size, num_layers=1, batch_first=True)
    self.decoder = torch.nn.Linear(hidden_size, N+1)
    self.hidden = None

  def forward(self, x):
    # inputs: x - input tensor of shape (batch_size, seq_length, N+1)
    # returns:
    # logits (scores for softmax) of shape (batch size, seq_length, N + 1)
    # TODO implement forward pass
    output, _ = self.gru(x)
    logits = self.decoder(output)
    return logits

  @torch.no_grad()
  def test_run(self, s):
    # This function accepts one string s containing lowercase characters a-z.
    # You need to map those characters to one-hot encodings,
    # then get the result from your network, and then convert the output
    # back to a string of the same length, with 0 mapped to ' ',
    # and 1-26 mapped to a-z.

    # TODO
    s = [(ord(c) - ord('a') + 1) for c in s]
    onehot = idx_to_onehot(torch.tensor(s, dtype=torch.int64)).unsqueeze(0).to(next(self.parameters()).device)
    logits = self.forward(onehot)
    pred = torch.argmax(logits, dim=2)
    pred = pred.squeeze(0)

    pred_str = ''.join([' ' if c == 0 else chr(c + ord('a') - 1) for c in pred])
    return pred_str




## Training
Below you need to implement the training of the model. We give you more freedom as for the implementation. The two limitations are that it has to execute within 10 minutes, and that error rate should be below 1%.

In [4]:
D=4
def test_model(model, sequence_length=15):
  """
  This is the test function that runs 100 different strings through your model,
  and checks the error rate.
  """
  total = 0
  correct = 0
  for i in range(500):
    s = ''.join([random.choice(string.ascii_lowercase) for i in range(random.randint(15, 25))])
    result = model.test_run(s)
    for c1, c2 in zip(s[:-D], result[D:]):
      correct += int(c1 == c2)
    total += len(s) - D

  return correct / total

In [5]:
import time
import torch
from torch.utils.data import DataLoader
# Assuming GRUMemory, EchoDataset, and idx_to_onehot are defined elsewhere

start_time = time.time()

gru_model = GRUMemory(hidden_size=128)  # Use 'gru_model' instead of 'model' to avoid confusion
optimizer = torch.optim.Adam(gru_model.parameters(), lr=0.01)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=10, gamma=0.1)
criterion = torch.nn.CrossEntropyLoss()  # Use 'criterion' for the loss function

ds = EchoDataset(delay=4, size=10000)
train_dataloader = DataLoader(ds, batch_size=64, num_workers=2, shuffle=False)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
gru_model.to(device)

epoch = 0
test_acc = 0

while test_acc <= 0.99:
    gru_model.train()
    for x, y in train_dataloader:
        x, y = x.to(device), y.to(device)
        x_onehot, y_onehot = idx_to_onehot(x), idx_to_onehot(y)

        optimizer.zero_grad()
        logits = gru_model(x_onehot)[:, 4:, :]
        y_target = y[:, 4:]
        loss_value = criterion(logits.reshape(-1, logits.size(-1)), y_target.reshape(-1))
        loss_value.backward()
        torch.nn.utils.clip_grad_norm_(gru_model.parameters(), 0.01)
        optimizer.step()

    scheduler.step()
    test_acc = test_model(gru_model)
    print(f'Epoch {epoch}: Test Accuracy: {test_acc}, Loss: {loss_value.item()}')
    epoch += 1

    # Time and accuracy checks
    duration = time.time() - start_time
    assert duration < 600, f'Execution took {duration:.2f} seconds, which is longer than 10 mins'
    assert test_acc > 0.99, f'Accuracy is too low, got {test_acc}, need 0.99'

print('Tests passed')



Epoch 0: Test Accuracy: 0.9926194645984489, Loss: 0.022024942561984062
Tests passed


## Variable delay model

Now, to make this more complicated, we want to have varialbe delay. So, now, the goal is to transform a sequence of pairs (character, delay) into a character sequence with given delay. Delay is constant within one sequence.

### Dataset
As before, we first implement the dataset:

In [6]:
class VariableDelayEchoDataset(torch.utils.data.IterableDataset):

  def __init__(self, max_delay=8, seq_length=20, size=1000):
    self.max_delay = max_delay
    self.seq_length = seq_length
    self.size = size

  def __len__(self):
    return self.size

  def __iter__(self):
    for _ in range(self.size):
      seq = torch.tensor([random.choice(range(1, N + 1)) for i in range(self.seq_length)], dtype=torch.int64)
      delay = random.randint(0, self.max_delay)
      result = torch.cat((torch.zeros(delay), seq[:self.seq_length - delay])).type(torch.int64)
      yield seq, delay, result

### Model

And the model.

In [17]:
class VariableDelayGRUMemory(torch.nn.Module):
    def __init__(self, hidden_size, max_delay):
        super().__init__()
        self.max_delay = max_delay
        self.gru = torch.nn.GRU(hidden_size, hidden_size, batch_first=True)
        self.fc = torch.nn.Linear(hidden_size, N + 1)
        self.embed = torch.nn.Embedding(N + 1, hidden_size//2)
        self.delay_embed = torch.nn.Embedding(max_delay + 1, hidden_size//2)
        # self.dropout = torch.nn.Dropout(p=0.2)

    def forward(self, x, delays):
        x = self.embed(x)
        delays = self.delay_embed(delays).unsqueeze(1)
        delays = delays.repeat(1, x.shape[1], 1)
        x = torch.cat((delays, x), dim=2)

        x, _ = self.gru(x)

        # 应用 Dropout
        # x = self.dropout(x)

        x = self.fc(x)
        return x

    @torch.no_grad()
    def test_run(self, s, delay):
    # This function accepts one string s containing lowercase characters a-z,
    # and a delay - the desired output delay.
    # You need to map those characters to one-hot encodings,
    # then get the result from your network, and then convert the output
    # back to a string of the same length, with 0 mapped to ' ',
    # and 1-26 mapped to a-z.

    # TODO
      idx = [(ord(c) - ord('a') + 1) for c in s]
      idx_tensor = torch.tensor(idx, dtype=torch.long).unsqueeze(0)  # Convert list of indices to a long tensor and add batch dimension

      delay_tensor = torch.tensor([delay], dtype=torch.long)  # Make sure delay is also a long tensor

      logits = self.forward(idx_tensor, delay_tensor)  # Pass the indices and delays to the model
      pred = torch.argmax(logits, dim=2)  # Get predictions
      pred = pred.squeeze(0)  # Remove batch dimension for comparison

      # Convert predictions back to string
      pred_str = ''.join([' ' if c == 0 else chr(c + ord('a') - 1) for c in pred])

      return pred_str


### Train

As before, you're free to do what you want, as long as training finishes within 10 minutes and accuracy is above 0.99 for delays between 0 and 8.

In [18]:
def test_variable_delay_model(model, seq_length=20):
  """
  This is the test function that runs 100 different strings through your model,
  and checks the error rate.
  """
  total = 0
  correct = 0
  for i in range(500):
    s = ''.join([random.choice(string.ascii_lowercase) for i in range(seq_length)])
    d = random.randint(0, model.max_delay)
    result = model.test_run(s, d)
    if d > 0:
      z = zip(s[:-d], result[d:])
    else:
      z = zip(s, result)
    for c1, c2 in z:
      correct += int(c1 == c2)
    total += len(s) - d

  return correct / total

In [24]:

import time
import torch
from torch.utils.data import DataLoader

# 记录开始时间
start_time = time.time()

# 初始化参数
N = 26
hidden_size = 128  # 增加隐藏层大小
MAX_DELAY = 8
SEQ_LENGTH = 20
BATCH_SIZE = 64  # 减小批次大小
LR = 0.003  # 更小的初始学习率
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

# 初始化模型
model = VariableDelayGRUMemory(hidden_size=hidden_size, max_delay=MAX_DELAY).to(device)
criterion = torch.nn.CrossEntropyLoss(reduction='none').to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=LR)
scheduler = torch.optim.lr_scheduler.MultiStepLR(optimizer, milestones=[100, 200, 250], gamma=0.8)

# 训练循环
epoch = 0
accuracy = 0
while accuracy <= 0.993:
    dataset = VariableDelayEchoDataset(max_delay=MAX_DELAY, seq_length=SEQ_LENGTH, size=2000)
    dataloader = DataLoader(dataset, batch_size=BATCH_SIZE)

    model.train()  # 确保模型处于训练模式
    for seq, delay_val, result in dataloader:
        seq, delay_val, result = seq.to(device), delay_val.to(device), result.to(device)
        optimizer.zero_grad()

        outputs = model(seq.long(), delay_val)  # 确保seq是长整型张量

        # 使用pad_sequence创建掩码
        mask = [torch.zeros(d).to(device) for d in delay_val]
        mask.append(torch.zeros(SEQ_LENGTH).to(device))
        mask = torch.nn.utils.rnn.pad_sequence(mask, padding_value=1, batch_first=True)[:-1]

        outputs = outputs.reshape(-1, outputs.size(-1))
        loss = criterion(outputs, result.reshape(-1))
        loss = loss.reshape(-1, SEQ_LENGTH)
        loss = torch.mean(loss * mask, dim=1)
        loss = loss.mean()

        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), 0.5)  # 更小的梯度裁剪值
        optimizer.step()

    scheduler.step()  # 更新学习率
    model.eval()  # 确保模型处于评估模式
    accuracy = test_variable_delay_model(model)
    print(f"Epoch {epoch}, Accuracy: {accuracy:.4f}, Loss: {loss.mean().item():.4f}")
    epoch += 1

# 断言和测试
end_time = time.time()

assert end_time - start_time < 600, 'Executing took longer than 10 mins'
assert test_variable_delay_model(model) > 0.99, 'Accuracy is too low'
print('Tests passed')


Epoch 0, Accuracy: 0.2983, Loss: 1.5510
Epoch 1, Accuracy: 0.4279, Loss: 1.4683
Epoch 2, Accuracy: 0.4915, Loss: 1.0233
Epoch 3, Accuracy: 0.5237, Loss: 1.1110
Epoch 4, Accuracy: 0.5588, Loss: 1.1992
Epoch 5, Accuracy: 0.5686, Loss: 0.8898
Epoch 6, Accuracy: 0.5772, Loss: 0.7159
Epoch 7, Accuracy: 0.6214, Loss: 0.6829
Epoch 8, Accuracy: 0.6219, Loss: 0.6293
Epoch 9, Accuracy: 0.6524, Loss: 0.7268
Epoch 10, Accuracy: 0.6657, Loss: 0.6013
Epoch 11, Accuracy: 0.6913, Loss: 0.7652
Epoch 12, Accuracy: 0.6838, Loss: 0.8965
Epoch 13, Accuracy: 0.7116, Loss: 0.6492
Epoch 14, Accuracy: 0.7170, Loss: 0.5569
Epoch 15, Accuracy: 0.7110, Loss: 0.4655
Epoch 16, Accuracy: 0.7321, Loss: 0.6589
Epoch 17, Accuracy: 0.7178, Loss: 0.4243
Epoch 18, Accuracy: 0.7487, Loss: 0.4900
Epoch 19, Accuracy: 0.7721, Loss: 0.5702
Epoch 20, Accuracy: 0.7626, Loss: 0.3039
Epoch 21, Accuracy: 0.7755, Loss: 0.5330
Epoch 22, Accuracy: 0.7698, Loss: 0.4977
Epoch 23, Accuracy: 0.7845, Loss: 0.5405
Epoch 24, Accuracy: 0.7898